<a href="https://colab.research.google.com/github/botondkozman/tracker_application/blob/colab/formrecognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Input parameters

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import firebase


In [17]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

db = None

def initialize_firebase():
  if not firebase_admin._apps:
    cred = credentials.Certificate('/content/drive/MyDrive/credential/odin-demo-2c3f0-firebase-adminsdk-dbjmc-732853d4fe.json')
    firebase_admin.initialize_app(cred)

def initialize_firestore():
  db = firestore.client()
  return db

def get_collection_data(collection_name):
    collection_ref = db.collection(collection_name)
    docs = collection_ref.stream()
    data = {}
    for doc in docs:
        data[doc.id] = doc.to_dict()
    return data

def add_data(collection, document, field):
    doc_ref = db.collection(collection).document(document)
    doc_ref.set(field)

def update_data(collection, document, field):
    doc_ref = db.collection(collection).document(document)
    doc_ref.update(field)

# Clone YOLOv5 and download requirements.

In [3]:
import os
import subprocess
import torch

folder_path = 'yolov5'
%cd /content

if os.path.exists(folder_path) and os.path.isdir(folder_path):
    print(f"The folder '{folder_path}' exists.")
else:
  subprocess.run(['git', 'clone', 'https://github.com/ultralytics/yolov5.git'])
  %cd yolov5
  subprocess.run(['pip', 'install', '-r', 'requirements.txt'])
  model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

/content
/content/yolov5


/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-10-24 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 219MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


# Deleting files from folder

In [4]:
import shutil

def delete_files_and_folders(folder_path):
  if os.path.exists(folder_path) and os.path.isdir(folder_path):
    files = os.listdir(folder_path)
    for file in files:
        file_path = os.path.join(folder_path, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)

        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
  else:
    print(f"The folder '{folder_path}' does not exist or is not a directory.")

# Loading the image

In [22]:
import requests
import os
from google.colab.patches import cv2_imshow

def load_image():
  folder_path = './data/images/'
  delete_files_and_folders(folder_path)

  image = cv2.imread(image_url)
  size = (image_size[0], image_size[1])
  image = cv2.resize(image, size)
  cv2.imwrite(folder_path + image_name_with_extension, image)

# Running YOLOv5

In [19]:
def run_yolov5():
  delete_files_and_folders(output_path)
  !python detect.py --source {source} --weights yolov5s.pt --conf 0.05 --save-txt --save-conf --project {output_path} --name {output_name}


# Display detected image

In [7]:
import matplotlib.pyplot as plt
import cv2

def display_image():
  path = output_path + '/' + output_name + '/' + image_name
  result_img = cv2.imread(path)
  plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.show()

# Extraction of labels into JSON array

In [8]:
import json

def extract_labels():
  first_values = []
  json_array = []

  labels = model.names
  with open(output_path + '/' + output_name + '/labels/' + output_name + '.txt', 'r') as file:
      lines = file.readlines()

      for line in lines:
          line = line.strip()
          if line:
              first_values.append(line.split()[0])
              json_array.append({'label': labels[int(line.split()[0])],
                                'rectangle': [float(line.split()[1]), float(line.split()[2]), float(line.split()[3]), float(line.split()[4])]})

  return json_array

Dominant color recognition

In [9]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
from collections import Counter

color_palette = {
    'red': [255, 0, 0],
    'green': [0, 255, 0],
    'blue': [0, 0, 255],
    'yellow': [255, 255, 0],
    'cyan': [0, 255, 255],
    'magenta': [255, 0, 255],
    'black': [0, 0, 0],
    'white': [255, 255, 255],
    'gray': [128, 128, 128],
    'purple': [128, 0, 128],
    'orange': [255, 165, 0],
    'pink': [255, 192, 203],
    'brown': [165, 42, 42],
    'lime': [0, 255, 0],
    'navy': [0, 0, 128],
    'olive': [128, 128, 0],
    'teal': [0, 128, 128],
    'maroon': [128, 0, 0],
    'silver': [192, 192, 192],
    'gold': [255, 215, 0]
}

palette_rgb = np.array(list(color_palette.values()), dtype=np.uint8)

def quantize_image(image, palette):
    pixels = image.reshape(-1, 3)
    kmeans = KMeans(n_clusters=len(palette), random_state=0).fit(pixels)
    labels = kmeans.predict(pixels)
    quantized_image = palette[labels].reshape(image.shape)
    return quantized_image, labels

def get_dominant_color(labels, color_names):
    label_counts = Counter(labels)
    dominant_label = label_counts.most_common(1)[0][0]
    dominant_color_name = color_names[dominant_label]
    return dominant_color_name, dominant_label

def get_dominant_color_rgb():
  image = cv2.imread(image_url)
  size=(250, 250)
  quantized_image, labels = quantize_image(image, palette_rgb)
  quantized_image=cv2.resize(quantized_image, size)
  height, width, channels = quantized_image.shape
  color_names = list(color_palette.keys())
  dominant_color, dominant_label = get_dominant_color(labels, color_names)
  return dominant_color

In [23]:
image_name = None
image_url = None
image_name_with_extension = None
output_path = 'runs/detect/exp'
output_name = image_name
source = None
image_size = None

if __name__ == "__main__":
  initialize_firebase()
  db = initialize_firestore()
  docs = get_collection_data('picture')
  for doc in docs:
    print(doc)
    image_name = doc
    image_url = '/content/drive/MyDrive/' + (str)(docs.get(doc).get("path"))
    image_name_with_extension = image_name + '.' + docs.get(doc).get("path").split(".")[1]
    output_name = image_name
    source = 'data/images/' + image_name_with_extension
    image_size = docs.get(doc).get("size")

    load_image()
    run_yolov5()
    json_array = extract_labels()
    dominant_color = get_dominant_color_rgb()

    update_data('picture', image_name, {'objects': json_array,
              'dominant_color': dominant_color})

image_4
output_path: runs/detect/exp
output_name: image_4
source: data/images/image_4.png
detect: weights=['yolov5s.pt'], source=data/images/image_4.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.05, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect/exp, name=image_4, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-378-g2f74455a Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1 /content/yolov5/data/images/image_4.png: 352x640 1 person, 1 kite, 1 surfboard, 29.1ms
Speed: 0.4ms pre-process, 29.1ms inference, 699.5ms NMS per image at shape (1, 3, 640, 640)
R